In [69]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import future

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import label_binarize

from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier


from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

!pip install langdetect

In [70]:
from langdetect import detect

In [71]:
data_filtered=pd.read_csv("data1.csv")

In [72]:
data_filtered.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,rr2wwvue,c64f525d77bf8b2ae4439a875f1bd8eb197a0519,PMC,Association and Host Selectivity in Multi-Host...,10.1371/journal.pone.0000041,PMC1762347,17183670.0,cc-by,The distribution of multi-host pathogens over ...,2006-12-20,"Malpica, José M.; Sacristán, Soledad; Fraile, ...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/c64f525d77bf8b2ae4439...,document_parses/pmc_json/PMC1762347.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
1,1n0rg5vd,6af02873a430f5f75dc56a917f26feaea89b8e79,PMC,The Effectiveness of Contact Tracing in Emergi...,10.1371/journal.pone.0000012,PMC1762362,17183638.0,cc-by,BACKGROUND: Contact tracing plays an important...,2006-12-20,"Klinkenberg, Don; Fraser, Christophe; Heesterb...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/6af02873a430f5f75dc56...,document_parses/pmc_json/PMC1762362.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
2,nzh87aux,9b7a0ad7b6c7f59e7a6cf1dc9d07912a273d19b5,PMC,The Waiting Time for Inter-Country Spread of P...,10.1371/journal.pone.0000143,PMC1764036,17206278.0,cc-by,BACKGROUND: The time delay between the start o...,2007-01-03,"Caley, Peter; Becker, Niels G.; Philp, David J.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/9b7a0ad7b6c7f59e7a6cf...,document_parses/pmc_json/PMC1764036.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
3,x2zq0lnt,d882f79283751d6c7f2e48cb991ee8abbcd913ae,PMC,The Transmissibility of Highly Pathogenic Avia...,10.1371/journal.pone.0000349,PMC1831494,17406673.0,cc-by,BACKGROUND: With the increased occurrence of o...,2007-04-04,"Garske, Tini; Clarke, Paul; Ghani, Azra C.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/d882f79283751d6c7f2e4...,document_parses/pmc_json/PMC1831494.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
4,s4y6uxsb,305cf1019ca9c81126b2e527fedf79969c2c522f,PMC,"ElaD, a Deubiquitinating Protease Expressed by...",10.1371/journal.pone.0000381,PMC1847702,17440617.0,cc-by,BACKGROUND: Ubiquitin and ubiquitin-like prote...,2007-04-18,"Catic, André; Misaghi, Shahram; Korbel, Gregor...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/305cf1019ca9c81126b2e...,document_parses/pmc_json/PMC1847702.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN


In [73]:
data_filtered['journal'].value_counts()

bioRxiv     4773
PLoS One    4398
BMJ         4246
Sci Rep     2057
Nature      2015
Name: journal, dtype: int64

In [74]:
data_filtered.shape

(17489, 19)

In [75]:
type(data_filtered)

pandas.core.frame.DataFrame

In [76]:
def CleanText(raw_text, remove_stopwords=False, stemming=False, split_text=False):
    '''
    Convert a raw review to a cleaned review
    '''
    text = BeautifulSoup(raw_text, 'lxml').get_text()  #remove html
    letters_only = re.sub("[^a-zA-Z]", " ", text)  # remove non-character
    words = letters_only.lower().split() # convert to lower case 
    
    if remove_stopwords: # remove stopword
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    if stemming==True: # stemming
        #stemmer = PorterStemmer()
        stemmer = SnowballStemmer('english') 
        words = [stemmer.stem(w) for w in words]
        
    if split_text==True:  # split text
        return (words)
    
    return( " ".join(words))

In [77]:
import warnings
warnings.filterwarnings("ignore")
cleaned_title=[]
for i in data_filtered['title']:
    cleaned_title.append(CleanText(i))
data_filtered['Cleaned title']=cleaned_title
data_filtered.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,Cleaned title
0,rr2wwvue,c64f525d77bf8b2ae4439a875f1bd8eb197a0519,PMC,Association and Host Selectivity in Multi-Host...,10.1371/journal.pone.0000041,PMC1762347,17183670.0,cc-by,The distribution of multi-host pathogens over ...,2006-12-20,"Malpica, José M.; Sacristán, Soledad; Fraile, ...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/c64f525d77bf8b2ae4439...,document_parses/pmc_json/PMC1762347.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,association and host selectivity in multi host...
1,1n0rg5vd,6af02873a430f5f75dc56a917f26feaea89b8e79,PMC,The Effectiveness of Contact Tracing in Emergi...,10.1371/journal.pone.0000012,PMC1762362,17183638.0,cc-by,BACKGROUND: Contact tracing plays an important...,2006-12-20,"Klinkenberg, Don; Fraser, Christophe; Heesterb...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/6af02873a430f5f75dc56...,document_parses/pmc_json/PMC1762362.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,the effectiveness of contact tracing in emergi...
2,nzh87aux,9b7a0ad7b6c7f59e7a6cf1dc9d07912a273d19b5,PMC,The Waiting Time for Inter-Country Spread of P...,10.1371/journal.pone.0000143,PMC1764036,17206278.0,cc-by,BACKGROUND: The time delay between the start o...,2007-01-03,"Caley, Peter; Becker, Niels G.; Philp, David J.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/9b7a0ad7b6c7f59e7a6cf...,document_parses/pmc_json/PMC1764036.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,the waiting time for inter country spread of p...
3,x2zq0lnt,d882f79283751d6c7f2e48cb991ee8abbcd913ae,PMC,The Transmissibility of Highly Pathogenic Avia...,10.1371/journal.pone.0000349,PMC1831494,17406673.0,cc-by,BACKGROUND: With the increased occurrence of o...,2007-04-04,"Garske, Tini; Clarke, Paul; Ghani, Azra C.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/d882f79283751d6c7f2e4...,document_parses/pmc_json/PMC1831494.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,the transmissibility of highly pathogenic avia...
4,s4y6uxsb,305cf1019ca9c81126b2e527fedf79969c2c522f,PMC,"ElaD, a Deubiquitinating Protease Expressed by...",10.1371/journal.pone.0000381,PMC1847702,17440617.0,cc-by,BACKGROUND: Ubiquitin and ubiquitin-like prote...,2007-04-18,"Catic, André; Misaghi, Shahram; Korbel, Gregor...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/305cf1019ca9c81126b2e...,document_parses/pmc_json/PMC1847702.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,elad a deubiquitinating protease expressed by ...


In [78]:
language = []
for i in data_filtered['Cleaned title']:
    language.append(detect(i))
data_filtered['Title Language']=language

In [79]:
data_filtered['Title Language'].value_counts()

en    17070
it      125
ca       75
fr       69
ro       31
es       29
nl       18
af       15
tl        8
pt        8
da        6
sv        6
cy        5
et        5
fi        3
lv        2
sl        2
so        2
id        2
sq        2
sw        1
no        1
hu        1
tr        1
hr        1
vi        1
Name: Title Language, dtype: int64

In [80]:
data_filtered['abstract']=data_filtered['abstract'].apply(str)

In [81]:
language = []
for i in data_filtered['abstract']:
    language.append(detect(i))
data_filtered['Abstract Language']=language

In [82]:
data_filtered['Abstract Language'].value_counts()

en    11965
tl     5523
ca        1
Name: Abstract Language, dtype: int64

In [83]:
data_filtered.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id', 'Cleaned title', 'Title Language', 'Abstract Language'],
      dtype='object')

In [84]:
filt = (data_filtered['Abstract Language'] =='en')

updated_df = data_filtered.loc[filt,['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id', 'Cleaned title', 'Title Language', 'Abstract Language']]

updated_df.to_csv('./data2.csv', index = False)

In [85]:
english_data_filtered = pd.read_csv("data2.csv")

In [86]:
english_data_filtered.shape

(11965, 22)

In [87]:
import spacy
from spacy import displacy
from collections import Counter
ner = spacy.load("en_core_web_lg")

In [88]:
## tag text
txt = english_data_filtered["abstract"].iloc[1]
doc = ner(txt)
## display result
spacy.displacy.render(doc, style="ent")

In [89]:
import collections

In [ ]:
## tag text and exctract tags into a list
english_data_filtered["tags"] = english_data_filtered["abstract"].apply(lambda x: [(tag.text, tag.label_) for tag in ner(x).ents] )
## utils function to count the element of a list
def utils_lst_count(lst):
    dic_counter = collections.Counter()
    for x in lst:
        dic_counter[x] += 1
    dic_counter = OrderedDict( 
                     sorted(dic_counter.items(), 
                     key=lambda x: x[1], reverse=True))
    lst_count = [ {key:value} for key,value in dic_counter.items() ]
    return lst_count

## count tags
english_data_filtered["tags"] = english_data_filtered["tags"].apply(lambda x: utils_lst_count(x))

## utils function create new column for each tag category
def utils_ner_features(lst_dics_tuples, tag):
    if len(lst_dics_tuples) > 0:
        tag_type = []
        for dic_tuples in lst_dics_tuples:
            for tuple in dic_tuples:
                type, n = tuple[1], dic_tuples[tuple]
                tag_type = tag_type + [type]*n
                dic_counter = Counter()
                for x in tag_type:
                    dic_counter[x] += 1
        return dic_counter[tag]
    else:
        return 0

## extract features
tags_set = []
for lst in english_data_filtered["tags"].tolist():
     for dic in lst:
            for k in dic.keys():
                tags_set.append(k[1])
tags_set = list(set(tags_set))
for feature in tags_set:
    english_data_filtered["tags_"+feature] = english_data_filtered["tags"].apply(lambda x: utils_ner_features(x, feature))

## print result
english_data_filtered.head()